In [1]:
import pandas as pd
df = pd.read_csv('mediapipe_motion_dataset.csv')
print(df.head())


   video_id  frame  keypoint_id           x           y  confidence  \
0         1      0            0  157.604113  210.127792         1.0   
1         1      0            1  171.454128  202.274399         1.0   
2         1      0            2  176.317885  203.785934         1.0   
3         1      0            3  180.936198  205.313911         1.0   
4         1      0            4  152.843964  195.391655         1.0   

   velocity_x  velocity_y  acceleration_x  acceleration_y  left_elbow_angle  \
0         NaN         NaN             NaN             NaN               NaN   
1         NaN         NaN             NaN             NaN               NaN   
2         NaN         NaN             NaN             NaN               NaN   
3         NaN         NaN             NaN             NaN               NaN   
4         NaN         NaN             NaN             NaN               NaN   

   right_elbow_angle  left_knee_angle  right_knee_angle  
0                NaN              NaN   

In [3]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df.fillna(df.select_dtypes(include=[np.number]).mean(), inplace=True)


scaler_0_10 = MinMaxScaler(feature_range=(0, 10))


weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
df['movement_score'] = np.abs(df[['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y', 'left_elbow_angle']]).dot(weights)


df['movement_score'] = df.groupby('video_id')['movement_score'].transform(lambda x: scaler_0_10.fit_transform(x.values.reshape(-1, 1)).flatten())


for joint, columns in {
    'right_elbow': ['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y', 'right_elbow_angle'],
    'left_elbow': ['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y', 'left_elbow_angle'],
    'right_knee': ['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y', 'right_knee_angle'],
    'left_knee': ['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y', 'left_knee_angle']
}.items():
    df[f'{joint}_score'] = np.abs(df[columns]).dot(weights)
    df[f'{joint}_score'] = df.groupby('video_id')[f'{joint}_score'].transform(lambda x: scaler_0_10.fit_transform(x.values.reshape(-1, 1)).flatten())


print(df[['movement_score', 'right_elbow_score', 'left_elbow_score', 'right_knee_score', 'left_knee_score']].head())


   movement_score  right_elbow_score  left_elbow_score  right_knee_score  \
0        0.000512           0.003908          0.000512          0.003771   
1        0.000512           0.003908          0.000512          0.003771   
2        0.000512           0.003908          0.000512          0.003771   
3        0.000512           0.003908          0.000512          0.003771   
4        0.000512           0.003908          0.000512          0.003771   

   left_knee_score  
0         0.000512  
1         0.000512  
2         0.000512  
3         0.000512  
4         0.000512  


In [5]:
print(df.head())

   video_id  frame  keypoint_id           x           y  confidence  \
0         1      0            0  157.604113  210.127792         1.0   
1         1      0            1  171.454128  202.274399         1.0   
2         1      0            2  176.317885  203.785934         1.0   
3         1      0            3  180.936198  205.313911         1.0   
4         1      0            4  152.843964  195.391655         1.0   

   velocity_x  velocity_y  acceleration_x  acceleration_y  left_elbow_angle  \
0   -0.127155   -0.199808        3.265679         4.03478        150.182844   
1   -0.127155   -0.199808        3.265679         4.03478        150.182844   
2   -0.127155   -0.199808        3.265679         4.03478        150.182844   
3   -0.127155   -0.199808        3.265679         4.03478        150.182844   
4   -0.127155   -0.199808        3.265679         4.03478        150.182844   

   right_elbow_angle  left_knee_angle  right_knee_angle  movement_score  \
0          150.94266   

In [7]:
video_scores = df.groupby('video_id')['movement_score'].agg(lambda x: (x.mean() + x.max()) / 2)


def dominant_side(df, joint1, joint2):
    dominant = df.groupby('video_id').apply(
        lambda x: joint1 if x[f'{joint1}_score'].mean() > x[f'{joint2}_score'].mean() else joint2
    )
    return dominant


dominant_elbow = dominant_side(df, 'right_elbow', 'left_elbow')
dominant_knee = dominant_side(df, 'right_knee', 'left_knee')


video_summary = pd.DataFrame({
    'movement_score': video_scores,
    'dominant_elbow': dominant_elbow,
    'dominant_knee': dominant_knee
})


print(video_summary)

          movement_score dominant_elbow dominant_knee
video_id                                             
1               5.071356    right_elbow    right_knee
2               5.104999     left_elbow     left_knee
3               5.115428     left_elbow    right_knee
4               5.064564     left_elbow     left_knee
5               5.080967     left_elbow     left_knee
6               5.125873    right_elbow    right_knee
7               5.160321    right_elbow     left_knee
8               5.079006    right_elbow     left_knee
9               5.064796     left_elbow     left_knee
10              5.154836    right_elbow     left_knee
11              5.078618     left_elbow    right_knee
12              5.053215     left_elbow     left_knee
13              5.350727     left_elbow    right_knee
14              5.129584     left_elbow    right_knee
15              5.211743    right_elbow    right_knee
16              5.179000    right_elbow     left_knee
17              5.050083    

In [9]:
stats_df = df.groupby('video_id').agg({
    'velocity_x': ['mean', 'std', 'min', 'max'],
    'velocity_y': ['mean', 'std', 'min', 'max'],
    'acceleration_x': ['mean', 'std', 'min', 'max'],
    'acceleration_y': ['mean', 'std', 'min', 'max'],
    'left_elbow_angle': ['mean', 'std', 'min', 'max'],
    'right_elbow_angle': ['mean', 'std', 'min', 'max'],
    'left_knee_angle': ['mean', 'std', 'min', 'max'],
    'right_knee_angle': ['mean', 'std', 'min', 'max'],
    'movement_score': ['mean', 'std', 'min', 'max']
})


stats_df.columns = ['_'.join(col).strip() for col in stats_df.columns.values]


video_summary = pd.DataFrame({
    'movement_score': df.groupby('video_id')['movement_score'].agg(lambda x: (x.mean() + x.max()) / 2),
    'dominant_elbow': dominant_elbow,
    'dominant_knee': dominant_knee
})

final_df = stats_df.join(video_summary, how='left')

# הצגת התוצאה
print(final_df.head())

          velocity_x_mean  velocity_x_std  velocity_x_min  velocity_x_max  \
video_id                                                                    
1               -0.093738       56.407784    -1146.894561      919.773969   
2                0.132434       64.890086     -972.379518     1118.765044   
3               -0.978978      146.426487    -3063.256931     1995.207524   
4                2.712626      388.133681    -6373.061371     7088.484192   
5                0.291111      144.415315    -5470.883703     2645.452309   

          velocity_y_mean  velocity_y_std  velocity_y_min  velocity_y_max  \
video_id                                                                    
1               -0.441762       89.294954    -2232.943802     2991.826954   
2               -1.170361       93.495174    -1641.715336     2465.295982   
3                0.452532      315.402829    -5464.594460     5462.217593   
4               -7.761311      448.112064   -14755.399990    10446.402884  

In [11]:
print(final_df.columns)
final_df.reset_index(inplace=True)
print(final_df.columns)
final_df.to_csv('mediapipe_videos_motion_dataset.csv')

Index(['velocity_x_mean', 'velocity_x_std', 'velocity_x_min', 'velocity_x_max',
       'velocity_y_mean', 'velocity_y_std', 'velocity_y_min', 'velocity_y_max',
       'acceleration_x_mean', 'acceleration_x_std', 'acceleration_x_min',
       'acceleration_x_max', 'acceleration_y_mean', 'acceleration_y_std',
       'acceleration_y_min', 'acceleration_y_max', 'left_elbow_angle_mean',
       'left_elbow_angle_std', 'left_elbow_angle_min', 'left_elbow_angle_max',
       'right_elbow_angle_mean', 'right_elbow_angle_std',
       'right_elbow_angle_min', 'right_elbow_angle_max',
       'left_knee_angle_mean', 'left_knee_angle_std', 'left_knee_angle_min',
       'left_knee_angle_max', 'right_knee_angle_mean', 'right_knee_angle_std',
       'right_knee_angle_min', 'right_knee_angle_max', 'movement_score_mean',
       'movement_score_std', 'movement_score_min', 'movement_score_max',
       'movement_score', 'dominant_elbow', 'dominant_knee'],
      dtype='object')
Index(['video_id', 'velocity_x_m

In [13]:
from sklearn.model_selection import train_test_split

video_ids = final_df['video_id'].unique()

train_videos, temp_videos = train_test_split(video_ids, test_size=0.3, random_state=42)

val_videos, test_videos = train_test_split(temp_videos, test_size=0.5, random_state=42)

train_df = final_df[final_df['video_id'].isin(train_videos)]
val_df = final_df[final_df['video_id'].isin(val_videos)]
test_df = final_df[final_df['video_id'].isin(test_videos)]

print(f"Train: {len(train_df)} samples, Validation: {len(val_df)} samples, Test: {len(test_df)} samples")



Train: 30 samples, Validation: 7 samples, Test: 7 samples


In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Without Movement score statistics
feature_cols = [
    'velocity_x_mean', 'velocity_x_std', 'velocity_x_min', 'velocity_x_max',
    'velocity_y_mean', 'velocity_y_std', 'velocity_y_min', 'velocity_y_max',
    'acceleration_x_mean', 'acceleration_x_std', 'acceleration_x_min', 'acceleration_x_max',
    'acceleration_y_mean', 'acceleration_y_std', 'acceleration_y_min', 'acceleration_y_max',
    'left_elbow_angle_mean', 'left_elbow_angle_std', 'left_elbow_angle_min', 'left_elbow_angle_max',
    'right_elbow_angle_mean', 'right_elbow_angle_std', 'right_elbow_angle_min', 'right_elbow_angle_max',
    'left_knee_angle_mean', 'left_knee_angle_std', 'left_knee_angle_min', 'left_knee_angle_max',
    'right_knee_angle_mean', 'right_knee_angle_std', 'right_knee_angle_min', 'right_knee_angle_max'
]

y_cols = ['movement_score', 'dominant_elbow', 'dominant_knee']

X_train, y_train = train_df[feature_cols], train_df[y_cols]
X_val, y_val = val_df[feature_cols], val_df[y_cols]
X_test, y_test = test_df[feature_cols], test_df[y_cols]

# Label encoding for categorical columns
elbow_encoder = LabelEncoder()
knee_encoder = LabelEncoder()

# Fit and transform the 'dominant_elbow' and 'dominant_knee' labels
y_train['dominant_elbow_encoded'] = elbow_encoder.fit_transform(y_train['dominant_elbow'])
y_train['dominant_knee_encoded'] = knee_encoder.fit_transform(y_train['dominant_knee'])
y_val['dominant_elbow_encoded'] = elbow_encoder.transform(y_val['dominant_elbow'])
y_val['dominant_knee_encoded'] = knee_encoder.transform(y_val['dominant_knee'])
y_test['dominant_elbow_encoded'] = elbow_encoder.transform(y_test['dominant_elbow'])
y_test['dominant_knee_encoded'] = knee_encoder.transform(y_test['dominant_knee'])





C:\Users\user\AppData\Local\Temp\ipykernel_584\637024732.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['dominant_elbow_encoded'] = elbow_encoder.fit_transform(y_train['dominant_elbow'])
C:\Users\user\AppData\Local\Temp\ipykernel_584\637024732.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['dominant_knee_encoded'] = knee_encoder.fit_transform(y_train['dominant_knee'])
C:\Users\user\AppData\Local\Temp\ipykernel_584\637024732.py:29: SettingWithCopyWarning: 
A value is trying to be 

In [17]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import pandas as pd

# אימון המודלים
movement_score_model_dt = DecisionTreeRegressor(random_state=42)
movement_score_model_dt.fit(X_train, y_train['movement_score'])

elbow_model_dt = DecisionTreeClassifier(random_state=42)
elbow_model_dt.fit(X_train, y_train['dominant_elbow_encoded'])

knee_model_dt = DecisionTreeClassifier(random_state=42)
knee_model_dt.fit(X_train, y_train['dominant_knee_encoded'])

# תחזיות עבור קבוצת הוולידציה
movement_score_preds = movement_score_model_dt.predict(X_val)
elbow_preds = elbow_model_dt.predict(X_val)
knee_preds = knee_model_dt.predict(X_val)

# הפיכת התחזיות המפוענחות חזרה לתוויות טקסט עבור המפרקים הדומיננטיים
elbow_preds_labels = elbow_encoder.inverse_transform(elbow_preds)
knee_preds_labels = knee_encoder.inverse_transform(knee_preds)

# יצירת DataFrame עם תוצאות ותחזיות
predictions_df = pd.DataFrame({
    'video_id': val_df['video_id'],
    'group': 'Validation',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_val['movement_score'],
    'predicted_movement_score': movement_score_preds,
    'true_dominant_elbow': y_val['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels,
    'true_dominant_knee': y_val['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels
})

# חישוב מדדים עבור קבוצת הוולידציה
mae = mean_absolute_error(y_val['movement_score'], movement_score_preds)
mse = mean_squared_error(y_val['movement_score'], movement_score_preds)
r2 = r2_score(y_val['movement_score'], movement_score_preds)

# הדפסת תוצאות
print("Validation Set Predictions:")
print(predictions_df)

print(f"Validation Set Mean Absolute Error: {mae}")
print(f"Validation Set Mean Squared Error: {mse}")
print(f"Validation Set R2 Score: {r2}")

# תחזיות עבור קבוצת האימון
movement_score_preds_train = movement_score_model_dt.predict(X_train)
elbow_preds_train = elbow_model_dt.predict(X_train)
knee_preds_train = knee_model_dt.predict(X_train)

elbow_preds_labels_train = elbow_encoder.inverse_transform(elbow_preds_train)
knee_preds_labels_train = knee_encoder.inverse_transform(knee_preds_train)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת האימון
train_predictions_df = pd.DataFrame({
    'video_id': train_df['video_id'],
    'group': 'Train',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_train['movement_score'],
    'predicted_movement_score': movement_score_preds_train,
    'true_dominant_elbow': y_train['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_train,
    'true_dominant_knee': y_train['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_train
})

# הצגת התוצאות של קבוצת האימון
print("Train Set Predictions:")
print(train_predictions_df)

# תחזיות עבור קבוצת המבחן
movement_score_preds_test = movement_score_model_dt.predict(X_test)
elbow_preds_test = elbow_model_dt.predict(X_test)
knee_preds_test = knee_model_dt.predict(X_test)

elbow_preds_labels_test = elbow_encoder.inverse_transform(elbow_preds_test)
knee_preds_labels_test = knee_encoder.inverse_transform(knee_preds_test)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת המבחן
test_predictions_df = pd.DataFrame({
    'video_id': test_df['video_id'],
    'group': 'Test',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_test['movement_score'],
    'predicted_movement_score': movement_score_preds_test,
    'true_dominant_elbow': y_test['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_test,
    'true_dominant_knee': y_test['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_test
})

# הצגת התוצאות של קבוצת המבחן
print("Test Set Predictions:")
print(test_predictions_df)

# הצגת מדדים לקבוצת המבחן
mae_test = mean_absolute_error(y_test['movement_score'], movement_score_preds_test)
mse_test = mean_squared_error(y_test['movement_score'], movement_score_preds_test)
r2_test = r2_score(y_test['movement_score'], movement_score_preds_test)

print(f"Test Set Mean Absolute Error: {mae_test}")
print(f"Test Set Mean Squared Error: {mse_test}")
print(f"Test Set R2 Score: {r2_test}")



Validation Set Predictions:
    video_id       group  true_movement_score  predicted_movement_score  \
4          5  Validation             5.080967                  5.068044   
6          7  Validation             5.160321                  5.079006   
12        13  Validation             5.350727                  5.211743   
19        20  Validation             5.096191                  5.068044   
24        25  Validation             5.149779                  5.227557   
34        35  Validation             5.221235                  5.122215   
36        37  Validation             5.180540                  5.079006   

   true_dominant_elbow predicted_dominant_elbow true_dominant_knee  \
4           left_elbow               left_elbow          left_knee   
6          right_elbow              right_elbow          left_knee   
12          left_elbow              right_elbow         right_knee   
19          left_elbow               left_elbow         right_knee   
24         right_elbo

In [19]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# אימון המודלים עם RandomForest
movement_score_model_rf = RandomForestRegressor(random_state=42)
movement_score_model_rf.fit(X_train, y_train['movement_score'])

elbow_model_rf = RandomForestClassifier(random_state=42)
elbow_model_rf.fit(X_train, y_train['dominant_elbow_encoded'])

knee_model_rf = RandomForestClassifier(random_state=42)
knee_model_rf.fit(X_train, y_train['dominant_knee_encoded'])

# תחזיות עבור קבוצת הוולידציה
movement_score_preds_rf = movement_score_model_rf.predict(X_val)
elbow_preds_rf = elbow_model_rf.predict(X_val)
knee_preds_rf = knee_model_rf.predict(X_val)

# הפיכת התחזיות המפוענחות חזרה לתוויות טקסט עבור המפרקים הדומיננטיים
elbow_preds_labels_rf = elbow_encoder.inverse_transform(elbow_preds_rf)
knee_preds_labels_rf = knee_encoder.inverse_transform(knee_preds_rf)

# יצירת DataFrame עם תוצאות ותחזיות
predictions_df_rf = pd.DataFrame({
    'video_id': val_df['video_id'],
    'group': 'Validation',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_val['movement_score'],
    'predicted_movement_score': movement_score_preds_rf,
    'true_dominant_elbow': y_val['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_rf,
    'true_dominant_knee': y_val['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_rf
})

# חישוב מדדים עבור קבוצת הוולידציה
mae_rf = mean_absolute_error(y_val['movement_score'], movement_score_preds_rf)
mse_rf = mean_squared_error(y_val['movement_score'], movement_score_preds_rf)
r2_rf = r2_score(y_val['movement_score'], movement_score_preds_rf)

# הדפסת תוצאות
print("Random Forest - Validation Set Predictions:")
print(predictions_df_rf)

print(f"Validation Set Mean Absolute Error (RF): {mae_rf}")
print(f"Validation Set Mean Squared Error (RF): {mse_rf}")
print(f"Validation Set R2 Score (RF): {r2_rf}")

# תחזיות עבור קבוצת האימון
movement_score_preds_train_rf = movement_score_model_rf.predict(X_train)
elbow_preds_train_rf = elbow_model_rf.predict(X_train)
knee_preds_train_rf = knee_model_rf.predict(X_train)

elbow_preds_labels_train_rf = elbow_encoder.inverse_transform(elbow_preds_train_rf)
knee_preds_labels_train_rf = knee_encoder.inverse_transform(knee_preds_train_rf)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת האימון
train_predictions_df_rf = pd.DataFrame({
    'video_id': train_df['video_id'],
    'group': 'Train',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_train['movement_score'],
    'predicted_movement_score': movement_score_preds_train_rf,
    'true_dominant_elbow': y_train['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_train_rf,
    'true_dominant_knee': y_train['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_train_rf
})

# הצגת התוצאות של קבוצת האימון
print("Random Forest - Train Set Predictions:")
print(train_predictions_df_rf)

# תחזיות עבור קבוצת המבחן
movement_score_preds_test_rf = movement_score_model_rf.predict(X_test)
elbow_preds_test_rf = elbow_model_rf.predict(X_test)
knee_preds_test_rf = knee_model_rf.predict(X_test)

elbow_preds_labels_test_rf = elbow_encoder.inverse_transform(elbow_preds_test_rf)
knee_preds_labels_test_rf = knee_encoder.inverse_transform(knee_preds_test_rf)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת המבחן
test_predictions_df_rf = pd.DataFrame({
    'video_id': test_df['video_id'],
    'group': 'Test',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_test['movement_score'],
    'predicted_movement_score': movement_score_preds_test_rf,
    'true_dominant_elbow': y_test['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_test_rf,
    'true_dominant_knee': y_test['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_test_rf
})

# הצגת התוצאות של קבוצת המבחן
print("Random Forest - Test Set Predictions:")
print(test_predictions_df_rf)

# הצגת מדדים לקבוצת המבחן
mae_test_rf = mean_absolute_error(y_test['movement_score'], movement_score_preds_test_rf)
mse_test_rf = mean_squared_error(y_test['movement_score'], movement_score_preds_test_rf)
r2_test_rf = r2_score(y_test['movement_score'], movement_score_preds_test_rf)

print(f"Test Set Mean Absolute Error (RF): {mae_test_rf}")
print(f"Test Set Mean Squared Error (RF): {mse_test_rf}")
print(f"Test Set R2 Score (RF): {r2_test_rf}")



Random Forest - Validation Set Predictions:
    video_id       group  true_movement_score  predicted_movement_score  \
4          5  Validation             5.080967                  5.089333   
6          7  Validation             5.160321                  5.123264   
12        13  Validation             5.350727                  5.164360   
19        20  Validation             5.096191                  5.097208   
24        25  Validation             5.149779                  5.133447   
34        35  Validation             5.221235                  5.122407   
36        37  Validation             5.180540                  5.124134   

   true_dominant_elbow predicted_dominant_elbow true_dominant_knee  \
4           left_elbow               left_elbow          left_knee   
6          right_elbow              right_elbow          left_knee   
12          left_elbow              right_elbow         right_knee   
19          left_elbow              right_elbow         right_knee   
24   

In [21]:
from sklearn.svm import SVR, SVC

# אימון המודלים עם SVR ו-SVC
movement_score_model_svr = SVR()
movement_score_model_svr.fit(X_train, y_train['movement_score'])

elbow_model_svc = SVC(random_state=42)
elbow_model_svc.fit(X_train, y_train['dominant_elbow_encoded'])

knee_model_svc = SVC(random_state=42)
knee_model_svc.fit(X_train, y_train['dominant_knee_encoded'])

# תחזיות עבור קבוצת הוולידציה
movement_score_preds_svr = movement_score_model_svr.predict(X_val)
elbow_preds_svc = elbow_model_svc.predict(X_val)
knee_preds_svc = knee_model_svc.predict(X_val)

# הפיכת התחזיות המפוענחות חזרה לתוויות טקסט עבור המפרקים הדומיננטיים
elbow_preds_labels_svc = elbow_encoder.inverse_transform(elbow_preds_svc)
knee_preds_labels_svc = knee_encoder.inverse_transform(knee_preds_svc)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת הוולידציה
predictions_df_svr = pd.DataFrame({
    'video_id': val_df['video_id'],
    'group': 'Validation',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_val['movement_score'],
    'predicted_movement_score': movement_score_preds_svr,
    'true_dominant_elbow': y_val['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_svc,
    'true_dominant_knee': y_val['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_svc
})

# חישוב מדדים עבור קבוצת הוולידציה
mae_svr = mean_absolute_error(y_val['movement_score'], movement_score_preds_svr)
mse_svr = mean_squared_error(y_val['movement_score'], movement_score_preds_svr)
r2_svr = r2_score(y_val['movement_score'], movement_score_preds_svr)

# הדפסת תוצאות
print("SVR - Validation Set Predictions:")
print(predictions_df_svr)

print(f"Validation Set Mean Absolute Error (SVR): {mae_svr}")
print(f"Validation Set Mean Squared Error (SVR): {mse_svr}")
print(f"Validation Set R2 Score (SVR): {r2_svr}")

# תחזיות עבור קבוצת האימון
movement_score_preds_train_svr = movement_score_model_svr.predict(X_train)
elbow_preds_train_svc = elbow_model_svc.predict(X_train)
knee_preds_train_svc = knee_model_svc.predict(X_train)

elbow_preds_labels_train_svc = elbow_encoder.inverse_transform(elbow_preds_train_svc)
knee_preds_labels_train_svc = knee_encoder.inverse_transform(knee_preds_train_svc)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת האימון
train_predictions_df_svr = pd.DataFrame({
    'video_id': train_df['video_id'],
    'group': 'Train',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_train['movement_score'],
    'predicted_movement_score': movement_score_preds_train_svr,
    'true_dominant_elbow': y_train['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_train_svc,
    'true_dominant_knee': y_train['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_train_svc
})

# הצגת התוצאות של קבוצת האימון
print("SVR - Train Set Predictions:")
print(train_predictions_df_svr)

# תחזיות עבור קבוצת המבחן
movement_score_preds_test_svr = movement_score_model_svr.predict(X_test)
elbow_preds_test_svc = elbow_model_svc.predict(X_test)
knee_preds_test_svc = knee_model_svc.predict(X_test)

elbow_preds_labels_test_svc = elbow_encoder.inverse_transform(elbow_preds_test_svc)
knee_preds_labels_test_svc = knee_encoder.inverse_transform(knee_preds_test_svc)

# יצירת DataFrame עם תוצאות ותחזיות עבור קבוצת המבחן
test_predictions_df_svr = pd.DataFrame({
    'video_id': test_df['video_id'],
    'group': 'Test',  # הוספת עמודה עבור קבוצת הנתונים
    'true_movement_score': y_test['movement_score'],
    'predicted_movement_score': movement_score_preds_test_svr,
    'true_dominant_elbow': y_test['dominant_elbow'],
    'predicted_dominant_elbow': elbow_preds_labels_test_svc,
    'true_dominant_knee': y_test['dominant_knee'],
    'predicted_dominant_knee': knee_preds_labels_test_svc
})

# הצגת התוצאות של קבוצת המבחן
print("SVR - Test Set Predictions:")
print(test_predictions_df_svr)

# הצגת מדדים לקבוצת המבחן
mae_test_svr = mean_absolute_error(y_test['movement_score'], movement_score_preds_test_svr)
mse_test_svr = mean_squared_error(y_test['movement_score'], movement_score_preds_test_svr)
r2_test_svr = r2_score(y_test['movement_score'], movement_score_preds_test_svr)

print(f"Test Set Mean Absolute Error (SVR): {mae_test_svr}")
print(f"Test Set Mean Squared Error (SVR): {mse_test_svr}")
print(f"Test Set R2 Score (SVR): {r2_test_svr}")



SVR - Validation Set Predictions:
    video_id       group  true_movement_score  predicted_movement_score  \
4          5  Validation             5.080967                  5.161909   
6          7  Validation             5.160321                  5.168355   
12        13  Validation             5.350727                  5.165936   
19        20  Validation             5.096191                  5.160747   
24        25  Validation             5.149779                  5.167111   
34        35  Validation             5.221235                  5.168323   
36        37  Validation             5.180540                  5.167530   

   true_dominant_elbow predicted_dominant_elbow true_dominant_knee  \
4           left_elbow              right_elbow          left_knee   
6          right_elbow              right_elbow          left_knee   
12          left_elbow              right_elbow         right_knee   
19          left_elbow              right_elbow         right_knee   
24         righ

In [23]:
import joblib

# שמירת המודלים
joblib.dump(movement_score_model_dt, 'mediapipe_movement_score_model_dt.pkl')
joblib.dump(elbow_model_dt, 'mediapipe_elbow_model_dt.pkl')
joblib.dump(knee_model_dt, 'mediapipe_knee_model_dt.pkl')

joblib.dump(movement_score_model_rf, 'mediapipe_movement_score_model_rf.pkl')
joblib.dump(elbow_model_rf, 'mediapipe_elbow_model_rf.pkl')
joblib.dump(knee_model_rf, 'mediapipe_knee_model_rf.pkl')

joblib.dump(movement_score_model_svr, 'mediapipe_movement_score_model_svr.pkl')
joblib.dump(elbow_model_svc, 'mediapipe_elbow_model_svc.pkl')
joblib.dump(knee_model_svc, 'mediapipe_knee_model_svc.pkl')

joblib.dump(elbow_encoder, 'mediapipe_elbow_encoder.pkl')
joblib.dump(knee_encoder, 'mediapipe_knee_encoder.pkl')


['mediapipe_knee_encoder.pkl']

In [25]:
import joblib
import os

models = [f for f in os.listdir(".") if f.endswith(".pkl")]  # חיפוש כל קובצי ה-PKL בתיקייה הנוכחית

for model_name in models:
    try:
        model = joblib.load(model_name)
        print(f" {model_name} was successfully loaded ")
    except Exception as e:
        print(f"Error loading {model_name}: {e}")

 elbow_encoder.pkl was successfully loaded 
 elbow_model_dt.pkl was successfully loaded 
 elbow_model_rf.pkl was successfully loaded 
 elbow_model_svc.pkl was successfully loaded 
 knee_encoder.pkl was successfully loaded 
 knee_model_dt.pkl was successfully loaded 
 knee_model_rf.pkl was successfully loaded 
 knee_model_svc.pkl was successfully loaded 
 mediapipe_elbow_encoder.pkl was successfully loaded 
 mediapipe_elbow_model_dt.pkl was successfully loaded 
 mediapipe_elbow_model_rf.pkl was successfully loaded 
 mediapipe_elbow_model_svc.pkl was successfully loaded 
 mediapipe_knee_encoder.pkl was successfully loaded 
 mediapipe_knee_model_dt.pkl was successfully loaded 
 mediapipe_knee_model_rf.pkl was successfully loaded 
 mediapipe_knee_model_svc.pkl was successfully loaded 
 mediapipe_movement_score_model_dt.pkl was successfully loaded 
 mediapipe_movement_score_model_rf.pkl was successfully loaded 
 mediapipe_movement_score_model_svr.pkl was successfully loaded 
 movement_score